# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [50]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [51]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\ssala\Class\World_Weather_Analysis\Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head()

,None,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Weather
0,0,Kabalo,CD,-6.0500,26.9167,70.45,96,100,4.14,Clouds
1,1,Kamenka,RU,51.3223,42.7678,28.18,96,100,8.86,Clouds
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,-25.60,68,75,5.75,Clouds
3,3,Umm Lajj,SA,25.0213,37.2685,71.71,62,81,5.68,Clouds
4,4,Nouadhibou,MR,20.9310,-17.0347,66.18,72,87,13.80,Clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [52]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [53]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]


# Display sample data
preferred_cities_df.head(10)

,None,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Weather
0,0,Kabalo,CD,-6.0500,26.9167,70.45,96,100,4.14,Clouds
3,3,Umm Lajj,SA,25.0213,37.2685,71.71,62,81,5.68,Clouds
5,5,Kindu,CD,-2.9500,25.9500,73.96,86,100,2.86,Clouds
6,6,Rikitea,PF,-23.1203,-134.9692,79.77,74,13,15.90,Clouds
7,7,Kapaa,US,22.0752,-159.3190,80.58,85,100,4.00,Rain
9,9,Cabo San Lucas,MX,22.8909,-109.9124,80.62,50,0,14.97,Clear
10,10,Atuona,PF,-9.8000,-139.0333,78.73,75,74,12.35,Clouds
12,12,San Rafael,AR,-34.6177,-68.3301,84.13,16,0,16.15,Clear
15,15,Portobelo,PA,9.5500,-79.6500,75.11,77,72,16.75,Clouds
21,21,Presidente Medici,BR,-11.1753,-61.9014,76.21,95,89,3.76,Clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [54]:
preferred_cities_df.count()

None            252
City            252
Country         252
Lat             252
Lng             252
Max Temp        252
Humidity        252
Cloudiness      252
Wind Speed      252
City Weather    252
dtype: int64

In [55]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()
clean_travel_cities_df = preferred_cities_df.dropna(how='all')
clean_travel_cities_df.rename(columns={'City Weather':'Current Description'}, inplace=True)

# Display sample data
clean_travel_cities_df.head()

,None,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kabalo,CD,-6.0500,26.9167,70.45,96,100,4.14,Clouds
3,3,Umm Lajj,SA,25.0213,37.2685,71.71,62,81,5.68,Clouds
5,5,Kindu,CD,-2.9500,25.9500,73.96,86,100,2.86,Clouds
6,6,Rikitea,PF,-23.1203,-134.9692,79.77,74,13,15.90,Clouds
7,7,Kapaa,US,22.0752,-159.3190,80.58,85,100,4.00,Rain


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [56]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [57]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kabalo,CD,70.45,Clouds,-6.0500,26.9167,
3,Umm Lajj,SA,71.71,Clouds,25.0213,37.2685,
5,Kindu,CD,73.96,Clouds,-2.9500,25.9500,
6,Rikitea,PF,79.77,Clouds,-23.1203,-134.9692,
7,Kapaa,US,80.58,Rain,22.0752,-159.3190,
9,Cabo San Lucas,MX,80.62,Clear,22.8909,-109.9124,
10,Atuona,PF,78.73,Clouds,-9.8000,-139.0333,
12,San Rafael,AR,84.13,Clear,-34.6177,-68.3301,
15,Portobelo,PA,75.11,Clouds,9.5500,-79.6500,
21,Presidente Medici,BR,76.21,Clouds,-11.1753,-61.9014,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [58]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [61]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Kabalo - nearest hotel: No hotel found
Umm Lajj - nearest hotel: No hotel found
Kindu - nearest hotel: No hotel found
Rikitea - nearest hotel: Chez Bianca & Benoit
Kapaa - nearest hotel: Pono Kai Resort
Cabo San Lucas - nearest hotel: Comfort Rooms
Atuona - nearest hotel: Pearl Resort
San Rafael - nearest hotel: Hotel Regional
Portobelo - nearest hotel: Luna Llena Hostal
Presidente Medici - nearest hotel: No hotel found
Ternate - nearest hotel: Boulivard ternate
Quatre Cocos - nearest hotel: Horisun
Luwuk - nearest hotel: No hotel found
Lazaro Cardenas - nearest hotel: Hotel Sol del Pacífico
Mahebourg - nearest hotel: Grand Bel Air
Butaritari - nearest hotel: No hotel found
Cape Town - nearest hotel: Townhouse Hotel
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Amahai - nearest hotel: No hotel found
Avarua - nearest hotel: Paradise Inn
San Patricio - nearest hotel: No hotel found
Tual - nearest hotel: Suita
Hilo - nearest hotel: Dolphin Bay Ho

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kabalo,CD,70.45,Clouds,-6.0500,26.9167,No hotel found
3,Umm Lajj,SA,71.71,Clouds,25.0213,37.2685,No hotel found
5,Kindu,CD,73.96,Clouds,-2.9500,25.9500,No hotel found
6,Rikitea,PF,79.77,Clouds,-23.1203,-134.9692,Chez Bianca & Benoit
7,Kapaa,US,80.58,Rain,22.0752,-159.3190,Pono Kai Resort
...,...,...,...,...,...,...,...
684,Port Keats,AU,77.31,Clouds,-14.2500,129.5500,No hotel found
685,Pauini,BR,74.73,Rain,-7.7136,-66.9764,No hotel found
689,San Fernando,PH,78.53,Clouds,15.0286,120.6898,Pampanga Lodge and Restaurant
690,Carutapera,BR,74.17,Clouds,-1.1950,-46.0200,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [62]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Rikitea,PF,79.77,Clouds,-23.1203,-134.9692,Chez Bianca & Benoit
7,Kapaa,US,80.58,Rain,22.0752,-159.3190,Pono Kai Resort
9,Cabo San Lucas,MX,80.62,Clear,22.8909,-109.9124,Comfort Rooms
10,Atuona,PF,78.73,Clouds,-9.8000,-139.0333,Pearl Resort
12,San Rafael,AR,84.13,Clear,-34.6177,-68.3301,Hotel Regional
...,...,...,...,...,...,...,...
672,Boa Vista,BR,85.98,Clouds,2.8197,-60.6733,Aipana Plaza Hotel
678,Sibolga,ID,75.25,Clouds,1.7427,98.7792,Hotel Prima Indah
683,Saint-Pierre,RE,81.91,Clear,-21.3393,55.4781,Tropic Hotel
689,San Fernando,PH,78.53,Clouds,15.0286,120.6898,Pampanga Lodge and Restaurant


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [67]:
# Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label=None)

clean_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Rikitea,PF,79.77,Clouds,-23.1203,-134.9692,Chez Bianca & Benoit
7,Kapaa,US,80.58,Rain,22.0752,-159.3190,Pono Kai Resort
9,Cabo San Lucas,MX,80.62,Clear,22.8909,-109.9124,Comfort Rooms
10,Atuona,PF,78.73,Clouds,-9.8000,-139.0333,Pearl Resort
12,San Rafael,AR,84.13,Clear,-34.6177,-68.3301,Hotel Regional


## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [87]:
# Configure the map plot
map_plot_1 = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoLight",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.9,
    color = "City",
    hover_cols=["Current Description","Country"],
    alpha=0.6  # set opacity to 0.6
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Current Description,Country)